In [ ]:
import matplotlib.pyplot as pl
import numpy as np
import typing

# An Introduction to Qiskit

Objectives
* Experiment with the Qiskit framework
* Build and execute quantum circuits
* Characterize and mitigate noise
* Measurements in different basis

## The Qiskit library

Open source framework is composed of 4 modules
* Terra
  * Composing quantum programs at the level of circuits and pulses with the code foundation
* Ignis
  * Addressing noise and errors
* Aer
  * Developing via simulators and noise models
* Aqua
  * Building algorithms and applications

Today (and for the whole workshop) we will be using classes defined in Terra, Ignis and Aer. 

In [ ]:
import qiskit

# Make sure that you have qiskit >= 0.23.1
qiskit.__qiskit_version__

## Bernstein-Vazirani algorithm

For the first part of this activity, we will use the [Bernstein-Vazirani algorithm](https://qiskit.org/textbook/ch-algorithms/bernstein-vazirani.html) to study quantum circuits with Qiskit. Note that this algorithm will not be used in the solution of the $H_2$ molecule!

A quick refresher on the Bernstein-Vazirani algorithm:
* A black-box function $f$ (the oracle), takes as input a string of $n$ bits $x = \{x_1, x_2, \ldots, x_n\}$, and returns the bitwise product of the input $x$ with some $n$-bits string $s$, $f(x) = x \cdot s \; (\text{mod } 2)$.
  * For example, if $s=011$ and $x=101$, then the oracle returns $(0\cdot1 + 1\cdot0 + 1\cdot1)\; (\text{mod } 2) = 1$
* The oracle returns the answer in an output bit (hence the oracle uses $n+1$ bits). 
* The objective is to find the string $s$.

Classically, this is achieved with $n$ calls to the oracle. With a quantum computer, we break the oracle's secret key with a single call to the oracle!

### Quantum algorithm

1. Initialise the inputs qubits to the $|0\rangle^{\otimes n}$ state, and the output qubit to $|−\rangle$.
2. Apply Hadamard gates to the input register
3. Query the oracle
4. Apply Hadamard gates to the input register
5. Measure

<img src="images/bernstein_vazirani_algo.svg"/>

# Quantum Circuits

Objectives

1. Build quantum circuits
2. Choose backend
3. Execute quantum circuits

## Building quantum circuits
* [QuantumRegister](https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumRegister.html#qiskit.circuit.QuantumRegister)
  * Collection of qubits
  * Indexed to reference individual qubit (e.g. `qr[0]`)

* [ClassicalRegister](https://qiskit.org/documentation/stubs/qiskit.circuit.ClassicalRegister.html#qiskit.circuit.ClassicalRegister)
  * Collection of bits
  * Used as the receiver of measurements on qubits

* [QuantumCircuit](https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.html#qiskit.circuit.QuantumCircuit)
  * Contains QuantumRegister and ClassicalRegister
  * Add gates specifying registers/qubits as arguments


### Bernstein-Vazirani circuit

Circuit for hidden string $s=011$ from the Qiskit textbook.<br>
Measure is made on the three input qubits ($q_0$, $q_1$ and $q_2$).

<img src="images/bernstein-vazirani_circuit.png">

In [ ]:
from qiskit import QuantumCircuit

def bv_oracle(secret_string:str) -> QuantumCircuit:
    """
    Build the oracle corresponding to the input string.

    Parameters
    ----------
    secret_string: str
        The hidden string, composed of '0's and/or '1'
    """

    s = secret_string[::-1] # index in string match qubit position
    secret_string_length = len(secret_string)

    qc = QuantumCircuit(secret_string_length + 1, name='oracle') # one extra qubit for the output

    for i in range(secret_string_length):
        if s[i] == '1':
            qc.cx(i, secret_string_length)

    return qc

In [ ]:
def bv_circuit(oracle: QuantumCircuit) -> QuantumCircuit:
    """
    Implementation of the Bernstein-Vazirani algorithm.

    Parameters
    ----------
    oracle: Instruction
        The oracle encoding the hidden string
    """
    # Number of qubits, including output qubit
    n = oracle.num_qubits
    
    # Initialize the QuantumCircuit
    qc = QuantumCircuit(n, n-1)
    
    # Put all qubits in superposition and add phase to the ancilla qubit
    qc.h(range(n))
    qc.z(n-1)
    
    # Add the oracle
    qc.append(oracle, [0,1,2,3]) # can be achieved with 
                                 # qc.compose(oracle, [0,1,2,3], inplace=True)

    # Restore input qubits
    qc.h(range(n-1))
    
    # Measure
    qc.measure(range(n-1), range(n-1))

    return qc

### Visualize the implementation

Let's see how our implementation of the Bernstein-Vazirani looks like.


In [ ]:
SECRET_STRING = '011'
'''
# The oracle corresponding to the secret_string
oracle011 = bv_oracle(SECRET_STRING)

# The quantum circuit implementing B-V algorithm
oracle_breaker_qc = bv_circuit(oracle011)
oracle_breaker_qc.draw()
'''

## Choosing a backend



### Providers

A [provider](https://quantum-computing.ibm.com/docs/manage/provider/) is defined by a hierarchical organization of hub, group, and project.

<img src="images/providers.png"  width="500">

In [ ]:
from qiskit import IBMQ

# IBMQ.save_account(TOKEN)
IBMQ.load_account()
IBMQ.providers()

The method `IBMQ.save_account(<TOKEN>)` will store your access token under `$HOME/.qiskit/qiskitrc`.

### Fair-share algorithm

When submitting jobs to the IBM Q Experience, the order which these jobs are executed is determined by a [fair-share queuing formula](https://quantum-computing.ibm.com/docs/manage/backends/queue/).

As the queue order is calculated dynamically as new jobs arrive, it is impossible to guarantee when a fair-share job will be executed.

A provider’s dynamic priority depends on how much of the provider’s allotted system time has been consumed over a given floating window of time (about 1 week). 

The fair-share algorithm works as follow:

1. Look at each hub in a device's queue and select the one with the largest dynamic priority at that time.
2. With the hub selected, repeat the same process but at the group level.
3. With hub and group selected, repeat one last time at the project level.
4. For the given provider, select the oldest job in queue from that provider.
5. Execute the job
6. Recompute the dynamic priorities for the providers by using the amount of time the job consumed on the system.

### Running jobs through the Sherbrooke hub

Advantages
* Devices with up to 63 qubits
* One quantum processor is assigned to Sherbrooke: *ibmq_sydney*. We have guarantee 5% use on this device.
* Reservations
* OpenPulse (other than _armonk_)

In [ ]:
# provider = IBMQ.get_provider(hub='ibm-q-sherbrooke')
provider = IBMQ.get_provider(hub='ibm-q-education')

### Choosing a backend

* Simulator  

    * QasmSimulator: Simulates experiments
      * Designed to mimic an actual device.
      * Executes `QuantumCircuit` for $N$ shots.
      * Returns count dictionary containing $N$ final values of `ClassicalRegister` in the circuit.
      * The circuit may contain gates, measure, reset, conditionals.
      * Can run on your machine ([qiskit.providers.aer.QasmSimulator](https://qiskit.org/documentation/stubs/qiskit.providers.aer.QasmSimulator.html)) and in the cloud ('*ibmq_qasm_simulator*').
      * Jobs sent to the *ibmq_qasm_simulator* are limited to run times under 10,000 seconds (~2.75 hours).

    * StatevectorSimulator: Simulates an ideal shot of an experiment (returns statevector)
      * Simulates a single-shot of a `QuantumCircuit` and returns the final quantum statevector.
      * The circuit may contain gates, and also measure, reset, and conditional operations.
      * Available locally ([qiskit.providers.aer.StatevectorSimulator](https://qiskit.org/documentation/stubs/qiskit.providers.aer.StatevectorSimulator.html)).

  * UnitarySimulator: Simulates an ideal circuit (return circuit unitary)
    * Constructs the full $n$-qubit unitary matrix for a QuantumCircuit.
    * Done by applying each gate matrix to an initial identity matrix.
    * Circuit may only contain gates.
    * Available locally ([qiskit.providers.aer.UnitarySimulator](https://qiskit.org/documentation/stubs/qiskit.providers.aer.UnitarySimulator.html)).

* Real device
  * Best practices
    * Debug circuits using simulators before going to hardware.
    * In general, it is preferable to use the smallest processor that provides the required capability and acceptable queue (though this information is not so relevant... see 'Fair-share queueing').
    * Use smaller number of shots for initial hardware runs (e.g., 1024 instead of 8192).
    * Overhead associated with preparing a job which can be significant for small circuits. Think at bundling circuits (more on this later).


In [ ]:
# List available backends for provider
provider.backends()

In [ ]:
simulator = provider.get_backend('ibmq_qasm_simulator')
bogota = provider.get_backend('ibmq_bogota')

### Backend properties

* Qubit's calibration data
* Gate's calibration data

You can access a description of the `BackendProperties` class [here](https://quantum-computing.ibm.com/docs/manage/backends/properties)


In [ ]:
bogota_prop = bogota.properties()
#bogota_prop.to_dict()

### Backend configuration

* Basis gates
* Coupling map
* Supported instructions
* Max experiments
* Max shots
* etc.

You can access a description of the `BackendConfiguration` class [here](https://quantum-computing.ibm.com/docs/manage/backends/configuration)

In [ ]:
bogota_conf = bogota.configuration()
#bogota_conf.to_dict()

### Experiment 1

Back to our Bernstein-Vazirani circuit, let's execute the circuit see if we can find the hidden string.

The `execute()` method can be [highly customized](https://qiskit.org/documentation/apidoc/execute.html).

We will start by leaving all parameters to their default values and by running the circuit on the *ibmq_qasm_simulator*.

In [ ]:
from qiskit import execute
from qiskit.visualization import plot_histogram
'''
job = execute(oracle_breaker_qc, simulator)

print(f'--------------------------------\nJob ID: {job.job_id()}\n--------------------------------')

# result() is blocking
counts = job.result().get_counts()
plot_histogram(counts)
'''

Awesome! We find the hidden string with a single execution (nb shots default to 1024).

Instead of using the blocking `result()` method, we can also retrieve a job using the ID of the job and the `IBMQBackendService.retrieve_job()` method.

In [ ]:
'''
counts = provider.backends.retrieve_job('5ffb6e77de0e058b1fa7e7a8').result().get_counts()
plot_histogram(counts)
'''

Since our algorithm works perfectly, let's try it on Bogota.

In [ ]:
# Modify this variable to use your job ID
BOGOTA_NAIVE_JOB_ID = "5fff524b20543fff779973fd"

In [ ]:
from qiskit.tools.monitor import job_monitor
'''
job = execute(oracle_breaker_qc, bogota)
BOGOTA_NAIVE_JOB_ID = job.job_id()

print(f'--------------------------------\nJob ID: {BOGOTA_NAIVE_JOB_ID}\n--------------------------------')

job_monitor(job)
'''

In [ ]:
counts = provider.backends.retrieve_job(BOGOTA_NAIVE_JOB_ID).result().get_counts()
plot_histogram(counts)

Oops, the probability to measure the secret string in down to 69.4%

In the next section, we will see different techniques we can try to improve our results.

# Errors

Current quantum processors are noisy and Qiskit offers different ways to model and study noisy on a quantum channel.

Objectives:
* Characterize the noise present on quantum devices
* Understand the role of the transpiler and the optimization process
* Mitigate noise present in quantum computations


## Calibration

<img src="images/bogota_calibration.png" width="500">

### Decoherence error

<img src="images/decoherence.png" width="600"><br>
Source: Krantz, P., _et al._ (2019). A quantum engineer’s guide to superconducting qubits. Applied Physics Reviews, 6(2). https://doi.org/10.1063/1.5089550

__Relaxation__

By convention, the higher energy state of the qubit is denoted $|1 \rangle$, so decoherence is the physical relaxation of the qubit from the $|1\rangle$ state to the $|0\rangle$ state (aka the ground state). 

A qubit prepared in the $|1\rangle$ state has probability $\mathcal{P}(|1\rangle) = \exp{(-t/T_1)}$ to be measured in the state $|1\rangle$ after time $t$. $T_1$ is the _energy relaxation time_.

__Dephasing__

Corresponds to the random change in the phase of a qubit.

Can be characterized by a Ramsay or a Hahn echo experiment from which we can extract the _dephasing time_ $T_2$.

<img src="images/t1_t2.png" width="500"><br>
Source: Krantz, P., _et al._ (2019). A quantum engineer’s guide to superconducting qubits. Applied Physics Reviews, 6(2). https://doi.org/10.1063/1.5089550

To learn more on decoherence with Qiskit, you can have a look at [this tutorial](https://qiskit.org/documentation/tutorials/noise/2_relaxation_and_decoherence.html) and [this section](https://qiskit.org/textbook/ch-quantum-hardware/calibrating-qubits-pulse.html) in the Qiskit textbook.

### Readout error

Refer to the classification error that can occur from kerneled measurement. 

<img src="images/calibrating-qubits.png" width="200"><br>
Source: https://qiskit.org/textbook/ch-quantum-hardware/calibrating-qubits-pulse.html


### Gate error

The calibration error of single-qubit ($SX$) and two-qubits ($CNOT$) gates is determined using randomized benchmarking.

This protocol consists in the generation of a sequence of $m$ gates with the particularity that the resulting circuit is the identity, for example

<img src="images/randomized_benchmarking_circuit.png" width="500">

We extract the gate errors from the RB curve.

<img src="images/randomized_benchmarking_results.png" width="300"><br>
Source: https://qiskit.org/textbook/ch-quantum-hardware/randomized-benchmarking.html


### Statistical error

Statistical error refers to the fact that a finite number of measurements is made for each experiment.

For $N$ measurements of some state $|\psi \rangle = \alpha |0\rangle + \beta |1 \rangle$, if the counts are $\{0: N_0, 1:N_1\}$ with $N_0 + N_1 = N$, then 

\begin{align}
\frac{N_{0}}{N} &\approx |\alpha|^{2}\\
\frac{N_{1}}{N} &\approx |\beta|^{2}
\end{align}

We can get a more accurate outcome of some quantum experiment by increasing the number of shots.

In [ ]:
# default is 1024
bogota_conf.max_shots

### Experiment 2

Let's go back to our Bernstein-Vazirani circuit and add a `NoiseModel` to our computation.


In [ ]:
from qiskit.providers.aer.noise import NoiseModel
'''
## Retreive the noise model corresponding to Bogota
bogota_nm = NoiseModel.from_backend(bogota_prop)

job = execute(oracle_breaker_qc,
              simulator,
              noise_model=bogota_nm)

counts = job.result().get_counts()
plot_histogram(counts)
'''

Not bad, we find the right string ~80% percent of the time.

This is indeed much better than the results we obtained with the real device (69%)...

Where this difference comes from?

### Experiment 3

To get closer to the real device, we can use the same topology and the same basis gates as the Bogota device. Let's do it!

In [ ]:
'''
job = execute(oracle_breaker_qc,
              simulator,
              noise_model=bogota_nm,
              coupling_map=bogota_conf.coupling_map,
              basis_gates=bogota_conf.basis_gates)

counts = job.result().get_counts()
plot_histogram(counts)
'''

So, using Bogota's topology and basis gates has a significant impact on the performance of our model and bring us pretty close to the results we obtained with the real device.

Wait, what is the actual circuit being executed?

### Transpiling

[Transpilation](https://qiskit.org/documentation/apidoc/transpiler.html) is the process of rewriting a given input circuit to match the topology of a specific quantum device, and/or to optimize the circuit for execution on present day noisy quantum systems.

<img src="images/transpiling_core_steps.png" width="500">


### Pass manager and optimization levels

The circuit transformations taking place during transpiling are known as [transpiler passes](https://qiskit.org/documentation/tutorials/circuits_advanced/04_transpiler_passes_and_passmanager.html).
The `PassManager` is responsible to chain together the different passes.

Qiskit define a variety of `Passe` that can be added to a custom `PassManager` object.
By setting the `optimization_level` argument in `execute()` and `transpile()` methods, we pick one of the pre-build pass manager defined in Qiskit.

* *optimization_level=0*: just maps the circuit to the backend, with no explicit optimization.
* *optimization_level=1*: maps the circuit, but also does light-weight optimizations by collapsing adjacent gates.
* *optimization_level=2*: medium-weight optimization, including a noise-adaptive layout and a gate-cancellation procedure based on gate commutation relationships.
* *optimization_level=3*: heavy-weight optimization, which in addition to previous steps, does resynthesis of two-qubit blocks of gates in the circuit.

By default, *optimization_level* is set to 1.

In [ ]:
from qiskit import transpile
from qiskit.transpiler import PassManager, passes
'''
qc = transpile(oracle_breaker_qc,
               coupling_map=bogota_conf.coupling_map,
               basis_gates=bogota_conf.basis_gates)
qc.draw()
'''

In [ ]:
print(f'Original circuit depth: {oracle_breaker_qc.depth()} - Transpiled circuit depth: {qc.depth()}')

What a mess!

Our circuit, which should have depth 5 turned to a depth 13.<br>
Moreover, instead of having 2 $\text{CNOT}$ gates, it has ~10 of them.

In the original circuit, the 2 $\text{CNOT}$ gates are applied to the qubits (0,3) and (1,3) respectively. But those qubits are not coupled!

In [ ]:
bogota_conf.coupling_map

In order to execute the circuit that we submit, the transpiler has to do 2 things:
* Insert $\text{SWAP}$ gates to "_move_" qubits around and be able to apply $\text{CNOT}$ gates to coupled qubits (_Note that the optimal swapping of qubits is an NP-hard task, hence heuristics are used to find such an assignment._)
* Since $\text{SWAP}$ gates do not belong to the basis gate set of Bogota, each $\text{SWAP}$ gate is expressed using three $\text{CNOT}$ gates.

A quick look should convince you that swapping qubits should not be necessary for this very simple circuit.

All we need to do is to assign qubits from our circuits to the physical qubits in a clever way!

One possible solution:
* virtual qubit $q_0 \rightarrow$ physical qubit 3
* virtual qubit $q_1 \rightarrow$ physical qubit 1
* virtual qubit $q_2 \rightarrow$ physical qubit 0
* virtual qubit $q_3 \rightarrow$ physical qubit 2

This mapping implied application of $CNOT$ gates between qubit pairs `[[1,2],[2,3]]`.

Is the **best solution**?

In [ ]:
'''
# Print CNOT error from Bogota calibration data
cx_errors = list(map(lambda cm: bogota_prop.gate_error("cx", cm), bogota_conf.coupling_map))
for i in range(len(bogota_conf.coupling_map)):
    print(f'      -> qubits {bogota_conf.coupling_map[i]} CNOT error: {cx_errors[i]}')
'''

Of course, the transpiler also translate the circuit to use the basis gates set of the targeted device

In [ ]:
bogota_conf.basis_gates

In [ ]:
'''
# Print U2 error from Bogota calibration data
u2_errors = list(map(lambda q: bogota_prop.gate_error("sx", q), range(bogota_conf.num_qubits)))
for i in range(bogota_conf.num_qubits):
    print(f'      -> qubits {i} U2 error: {u2_errors[i]}')
'''

By inspection, we see that we should avoid using CNOT gates between qubits {0,1} and aim for CNOT gates between `[[1,2],[2,3]]`.

Also, we find that $SX$ gate error is the largest for qubit 2.

Fair enough, our solution seems acceptable. Let's see what the circuit looks like with this mapping!

In [ ]:
'''
# when a list argument is used, virtual qubits are ordered
layout = [3,1,0,2]

qc_il = transpile(oracle_breaker_qc,
                  coupling_map=bogota_conf.coupling_map,
                  basis_gates=bogota_conf.basis_gates,
                  initial_layout=layout)
qc_il.draw()
'''

In [ ]:
print(f'Original circuit depth: {oracle_breaker_qc.depth()} - Transpiled circuit depth: {qc.depth()} - Transpiled circuit with physical qubit mapping depth: {qc_il.depth()}')

In [ ]:
# Now, compare circuit depth for different optimization levels
'''
qc_opt0 = transpile(oracle_breaker_qc,
                       coupling_map=bogota_conf.coupling_map,
                       basis_gates=bogota_conf.basis_gates,
                       initial_layout=layout,
                       optimization_level=0)
qc_opt1 = transpile(oracle_breaker_qc,
                       coupling_map=bogota_conf.coupling_map,
                       basis_gates=bogota_conf.basis_gates,
                       initial_layout=layout,
                       optimization_level=1)
qc_opt2 = transpile(oracle_breaker_qc,
                       coupling_map=bogota_conf.coupling_map,
                       basis_gates=bogota_conf.basis_gates,
                       initial_layout=layout,
                       optimization_level=2)
qc_opt3 = transpile(oracle_breaker_qc,
                       coupling_map=bogota_conf.coupling_map,
                       basis_gates=bogota_conf.basis_gates,
                       initial_layout=layout,
                       optimization_level=3)

print(f'Depth for transpiled circuits with optimization levels {0,1,2,3} respectively: ({qc_opt0.depth()}, {qc_opt1.depth()}, {qc_opt2.depth()}, {qc_opt3.depth()})')
'''

### Experiment 4

Execute our Bernstein-Vazirani circuit by specifying the mapping of the quantum circuit to the device with optimization level 1.

In [ ]:
'''
qc_opt1_result = execute(qc_opt1,
                         simulator,
                         noise_model=bogota_nm,
                         coupling_map=bogota_conf.coupling_map,
                         basis_gates=bogota_conf.basis_gates).result()

counts = qc_opt1_result.get_counts()
plot_histogram(counts)
'''

**Conclusion**

By carefully analysing our quantum circuit and choosing a logical to physical qubit mapping leading to an _error aware_ assignation, we can greatly improve the result of our experiments!

With around 80% for state 011, we are back to the results we obtained at Experiment 2.

## Error mitigation

We improved the output of our Bernstein-Vazirani circuit by cleverly mapping logical qubits to physical qubits... can we do better?

* Error correction<br>
  Out of reach with current devices having
  1. limited number of qubits
  2. limited connectivity between qubits
  3. limited coherence time
* Error mitigation<br>
  * Gate errors: hard to mitigate
  * Decoherence errors: should not be significant given our shallow circuit
  * Readout errors: let's try!

### Calibration matrix

As a simple model of the measurement noise, we can imagine that the measurement first selects one of these outputs in a perfect and noiseless manner, and then noise subsequently causes this perfect output to be perturbed before it is returned to the user.

\begin{equation}
R_{noisy} = MR_{ideal} \Leftrightarrow R_{ideal} = M^{-1}R_{noisy}
\end{equation}

where $R_\text{noisy}$ and $R_\text{ideal}$ stands for the ideal and noisy results respectively and $M$ is the perturbation matrix representing readout errors.

If we can approximate $M$, then we should be able to get closer to $R_\text{ideal}$.



The Ignis module provides all the [tooling for this purpose](https://qiskit.org/documentation/apidoc/mitigation.html#measurement) and a [tutorial](https://qiskit.org/documentation/tutorials/noise/3_measurement_error_mitigation.html) is also available in Qiskit.

The idea is to prepare all $2^n$ basis input states and compute the probability of measuring counts in the other basis states.

From these calibrations, it is possible to correct the average results of another experiment of interest.

In [ ]:
from qiskit.circuit import QuantumRegister
from qiskit.ignis.mitigation.measurement import complete_meas_cal
'''
# Generate the calibration circuits for the 3 qubits we measure
qr = QuantumRegister(3)
qubit_list = [0,1,2]

# meas_calibs is a list containing 2^n circuits, one for each state.
meas_calibs, state_labels = complete_meas_cal(qubit_list=qubit_list, qr=qr, circlabel='mcal')

print(f'Number of circuits: {len(meas_calibs)}')
meas_calibs[1].draw()
'''

### Circuit bundling

To compute the calibration matrix we will have to execute these $2^n$ circuits which, in our case, turns out to be 16.

There is some overhead with each individual job that is processed. So, when sending multiple jobs, this overhead adds up and negatively impact our priority in the fair-share queuing algorithm.

This is where **circuit bundling** comes in: it is possible include multiple circuits into one job.

To _bundle_ multiple circuits into one job, all you have to do is to submit a list of `QuantumCircuit` to the `execute()` method.

In [ ]:
# maximum number of circuits that can be bundled in a single job
bogota_conf.max_experiments

In [ ]:
from qiskit.ignis.mitigation.measurement import complete_meas_cal
'''
# since we measure only {q_0, q_1, q_2} we will compute the calibration matrix for these 3 qubits
calibration_layout = [3,1,0]

result = execute(meas_calibs,
                 simulator,
                 shots=8192,
                 noise_model=bogota_nm,
                 coupling_map=bogota_conf.coupling_map,
                 basis_gates=bogota_conf.basis_gates,
                 initial_layout=calibration_layout).result()
'''

We can then retrieve the counts for individual experiments

In [ ]:
# For example, plot histogram for circuit corresponding to state '001' (index 1)
plot_histogram(result.get_counts(meas_calibs[1]))

### Error mitigation with calibration matrix

Now that we know what circuit bundling is, we can use the result of the basis states experiments to mitigate the error of the Bernstein-Vazirani circuit.

In [ ]:
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
'''
# Initialize the measurement correction fitter for a full calibration
meas_fitter = CompleteMeasFitter(result, state_labels)
meas_fitter.plot_calibration()
'''

In [ ]:
'''
# Get the filter object
meas_filter = meas_fitter.filter

# Apply the calibration matrix to the results of our Bernstein-Vazirani circuit
counts = meas_filter.apply(qc_opt1_result).get_counts(0)
plot_histogram(counts)
'''

Impressive, by applying the inverse of the calibration matrix to our results we achieve 100% accuracy with the simulator using the Bogota noise model (remember we were around 85%)!

Let's try it with the real device!

In [ ]:
'''
BOGOTA_MEASCAL_JOB_ID = "6002116d25c4183b28234d45"
result = provider.backends.retrieve_job(BOGOTA_MEASCAL_JOB_ID).result()
'''

In [ ]:
'''
meas_fitter = CompleteMeasFitter(result, state_labels)
meas_fitter.plot_calibration()
'''

In [ ]:
'''
BOGOTA_EDUCATED_JOB_ID = "600223ed25c418d649234e02"
result = provider.backends.retrieve_job(BOGOTA_EDUCATED_JOB_ID).result()
meas_filter = meas_fitter.filter

# Apply the calibration matrix to the results of our Bernstein-Vazirani circuit
counts = meas_filter.apply(result).get_counts(0)
plot_histogram(counts)
'''

By applying this error mitigation strategy, we measure the expected state 89.4% of the time (recall that without error mitigation, we were at 79.7%)

# Measurement

Objectives:
* Understand what is a measurement in the computational basis states
* Perform measurements in different basis state

## Projective measurement

The measurement of an observable $M$ can be described by a collection ${P_m}$ of projectors onto the eigenspace of $M$ where $m$ refers to the measurement outcomes that may occur in the experiment

\begin{align}
M = \sum_m mP_m.
\end{align}

This is known as the spectral decomposition of the operator $M$.

If the state of the quantum system is $|\psi\rangle$ immediately before the measurement then the probability that result $m$ occurs:

\begin{align}
\mathcal{P}(m) = \langle \psi |P_m | \psi \rangle
\end{align}

## Computational basis states

Let's consider a qubit in an arbitrary state $|\psi \rangle = \alpha |0 \rangle + \beta |1 \rangle$ where the states $|0 \rangle$ and $|1 \rangle$ are known as **computational basis states**.

The Born's rule tells us that if we measure the qubit, we will obtain either '0' or '1' with probability $|\alpha|^2$ and $|\beta|^2$ respectively.

The projectors allowing us to measure the state of a qubit in the computational basis states are $P_0 = |0\rangle \langle 0|$ and $P_1 = |1\rangle \langle 1|$.

Therefore, the probability of measuring '0' is given by

\begin{align}
\mathcal{P}(0) &= \langle \psi |P_0 | \psi \rangle \\
&= (\alpha^\dagger  \langle 0| + \beta^\dagger \langle 1| ) \ |0\rangle \langle 0| \ (\alpha |0 \rangle + \beta |1 \rangle) \\
&= |\alpha|^2
\end{align}

## Measurement in other basis states

Given any basis states $|a\rangle$ and $|b\rangle$ for a qubit, it is possible to express an arbitrary state as a linear combination $\gamma |a\rangle + \delta |b\rangle$ of those states.

Furthermore, provided $|a\rangle$ and $|b\rangle$ are orthonormal, it is possible to perform a measurement with respect to the $|a\rangle$, $|b\rangle$ basis, giving the result $a$ with probability $|\gamma|^2$ and $b$ with probability $|\delta|^2$.

<p style="text-align: center; font-weight: bold;">But with a quantum computer, we only have access to measurements in the computational basis set</p>

To perform a measurement in another basis states set, we simply have to unitarily transform from the basis we wish to perform the measurement into the computational basis, then measure.

## Pauli measurements

### Pauli-Z
The basis states of Pauli-$Z$ observable correspond to the computational basis states, with eigenvalues $\{+1, -1\}$ respectively.

<table><tr>
<td> <img src="images/Bloch_Sphere.png" width="200"/> </td>
<td>
\begin{align}|\psi\rangle &= \alpha|0\rangle + \beta |1\rangle \;\, \text{where}\\
\alpha &= \cos(\frac{\theta}{2}) \\
\beta &= e^{i\phi}\sin(\frac{\theta}{2})\end{align}
</td>
</tr></table>



We have

\begin{align}
Z = \sum_{m_z \in \{\pm 1\}} m_zP_{m_z} = |0\rangle \langle 0| - |1\rangle \langle 1|
\end{align}

### Pauli-X
We use symbols $|+\rangle$ and $|-\rangle$ to define the basis states of the Pauli-$X$ observable, with eigenvalues $\{+1, -1\}$ respectively.

The $X$ basis states can be expressed in the computational basis states with the following unitary transform

\begin{align}
|+\rangle &= \cos(\frac{\pi}{4})|0\rangle + \sin(\frac{\pi}{4})|1\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}} \equiv H|0\rangle \\
|-\rangle &= \cos(\frac{\pi}{4})|0\rangle + e^{i\pi}\sin(\frac{\pi}{4})|1\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}} \equiv H|1\rangle
\end{align}
where $H$ is the Hadamard transform.

Therefore, a Pauli-$X$ measurement in the computational basis corresponds to 

\begin{align}
X &= \sum_{m_x \in \{\pm 1\}} m_xP_{m_x} \\
&= |+\rangle \langle +| - |-\rangle \langle -| \\
&= H|0\rangle \langle 0|H - H|1\rangle \langle 1|H
\end{align}

where we have used the fact that $H^\dagger = H$.

### Pauli-Y
We use symbols $|\circlearrowleft \rangle$ and $|\circlearrowright\rangle$ to define the basis states of the Pauli-$Y$ observable, with eigenvalues $\{+1, -1\}$ respectively.

The $Y$ basis states can be expressed in the computational basis states with the following unitary transform

\begin{align}
|\circlearrowleft\rangle &= \cos(\frac{\pi}{4})|0\rangle + e^{i\pi/2}\sin(\frac{\pi}{4})|1\rangle = \frac{|0\rangle + i|1\rangle}{\sqrt{2}} \equiv SH|0\rangle \\
|\circlearrowright\rangle &= \cos(\frac{\pi}{4})|0\rangle + e^{i3\pi/2}\sin(\frac{\pi}{4})|1\rangle = \frac{|0\rangle - i|1\rangle}{\sqrt{2}} \equiv SH|1\rangle
\end{align}
where the $S$ gate performs a rotation of $\phi = \frac{\pi}{2}$ around the Z-axis direction.

Therefore, a Pauli-$Y$ measurement in the computational basis corresponds to 

\begin{align}
Y &= \sum_{m_y \in \{\pm 1\}} m_yP_{m_y} \\
&= |\circlearrowleft\rangle \langle \circlearrowleft| - |\circlearrowright\rangle \langle \circlearrowright| \\
&= SH|0\rangle \langle 0|HS^\dagger - SH|1\rangle \langle 1|HS^\dagger.
\end{align}

### Identity

Measuring the identity operator is trivial since its eigenvectors are degenerate with eigenvalue $\{+1\}$.

\begin{align}
I = \sum_{m_z \in \{\pm 1\}} m_zP_{m_z} = |0\rangle \langle 0| + |1\rangle \langle 1|
\end{align}
which corresponds to the _completeness relation_.

## Measurement's average value

Since $\sum_m \mathcal{P}(m) = 1$, the average value of an observable $M$ is given by:

\begin{align}
\mathbb{E}(M) &= \sum_m m \mathcal{P}(m) \\
&= \sum_m m \langle \psi|P_m|\psi\rangle \\
&= \langle \psi| \big( \sum_m m P_m \big)|\psi\rangle \\
&= \langle \psi| M|\psi\rangle \\
&\equiv \langle M \rangle
\end{align}

Consider a general one-qubit state $|\psi \rangle = \alpha |0\rangle + \beta |1 \rangle$ and let's compute the average value of the three Pauli observables.

**Pauli-X**

\begin{align}
\langle X \rangle &= \langle \psi | \big (H|0\rangle \langle 0|H - H|1\rangle \langle 1|H \big ) | \psi \rangle \\
&= \langle \psi | H|0\rangle \langle 0|H |\psi \rangle - \langle \psi | H|1\rangle \langle 1|H |\psi \rangle \\
&= \langle \psi' |0\rangle \langle 0| \psi' \rangle - \langle \psi'|1\rangle \langle 1 |\psi'\rangle \\
&=  |\langle 0|\psi'\rangle |^2 -  | \langle 1|\psi' \rangle |^2
\end{align}

where $|\psi'\rangle = H|\psi\rangle$.

**Pauli-Y**

\begin{align}
\langle Y \rangle &= \langle \psi | \big (SH|0\rangle \langle 0|HS^\dagger - SH|1\rangle \langle 1|HS^\dagger \big ) | \psi \rangle \\
&= \langle \psi | SH|0\rangle \langle 0|HS^\dagger |\psi \rangle - \langle \psi | SH|1\rangle \langle 1|HS^\dagger |\psi \rangle \\
&= \langle \psi'' |0\rangle \langle 0| \psi'' \rangle - \langle \psi''|1\rangle \langle 1 |\psi''\rangle \\
&=  |\langle 0|\psi'' \rangle |^2 -  | \langle 1|\psi'' \rangle |^2
\end{align}

where $|\psi''\rangle = HS^\dagger|\psi\rangle$.

**Pauli-Z**

Finally,

\begin{align}
\langle Z \rangle &= \langle \psi | \big (|0\rangle \langle 0| - |1\rangle \langle 1| \big ) | \psi \rangle \\
&= \langle \psi |0\rangle \langle 0|\psi \rangle - \langle \psi |1\rangle \langle 1|\psi \rangle \\
&=  |\langle 0|\psi \rangle |^2 -  | \langle 1|\psi \rangle |^2
\end{align}

**Identity**

Of course, $\langle I \rangle = \langle \psi | \psi \rangle = 1$.

When running an experiment, we prepare a state $|\psi \rangle = \alpha |0\rangle + \beta |1 \rangle$ and measure the qubit.

If we repeat the same experiment for $N$ shots and the counts are $\{0: N_0, 1:N_1\}$ with $N_0 + N_1 = N$, then 

\begin{align}
p_{0} & = \frac{N_{0}}{N}\approx\left|\left\langle 0|\psi\right\rangle \right|^{2}=\left|\alpha\right|^{2}\\
p_{1}&=\frac{N_{1}}{N}\approx\left|\left\langle 1|\psi\right\rangle \right|^{2}=\left|\beta\right|^{2}. \\
\end{align}

## Measurement along an arbitrary axis

Suppose $\overrightarrow{v} = (v_x, v_y, v_z)$ is a three dimension unit vector with $\{v_x, v_y, v_z\} \in \Re$. We can define an observable

\begin{align}
\mathcal{O} = \overrightarrow{v} \cdot \overrightarrow{\sigma} = v_xX + v_yY + v_zZ
\end{align}

where $X$, $Y$ and $Z$ are the Pauli operators.

Measurement of this observable - a _measurement of spin along the $\overrightarrow{v}$ axis_ - correspond to

\begin{align}
\langle \mathcal{O} \rangle &= \langle v_xX + v_yY + v_zZ \rangle \\
&= v_x\langle X \rangle + v_y\langle Y \rangle + v_z\langle Z \rangle
\end{align}

## Summary

To measure a Pauli operator, one needs to apply the following unitary transformation to the qubit state and perform a measure in the computational basis states

Pauli measurement &nbsp; &nbsp; &nbsp; | Unitary transformation
:-|:-
X | $H$
Y | $HS^\dagger$
Z | $I$

## Let's code!

We will measure the average value of the Pauli operators for a randomly picked initial state.

In [ ]:
from qiskit.result import Counts

def one_qubit_pauli_average_from_counts(counts: Counts) -> float:
    """
    Compute the average of a measurement made in the Z-basis.

    In the Z-basis, the eigenvalues are +1 and -1 for the '0' state and
    '1' states respectively.

    Parameters
    ----------
    counts: Counts
        The counts that result from the execution of a job. 
    """
    
    Z_EIGENVALUES = np.array([1,-1])
    
    counts_vector = np.array([counts['0'], counts['1']])
    counts_vector = counts_vector / np.linalg.norm(counts_vector)
    
    return Z_EIGENVALUES.dot(counts_vector)

In [ ]:
from qiskit.extensions import Initialize
from qiskit.quantum_info import random_statevector
from qiskit.visualization import plot_bloch_multivector
'''
# Create random 1-qubit state
psi = random_statevector(2)
print(psi)
# Show it on a Bloch sphere
plot_bloch_multivector(psi)
'''

### Pauli-Z measurement with Qiskit

In [ ]:
'''
# Circuit with a single qubit
qc = QuantumCircuit(1)

# Initialization of the circuit with random state
qc.initialize(psi.data, [0])

# Measure Pauli-Z operator
qc.measure_all()
qc.draw()
'''

In [ ]:
counts = execute(qc, simulator).result().get_counts()
plot_histogram(counts)

In [ ]:
print(f'Pauli-Z measurement average for random state vector: {one_qubit_pauli_average_from_counts(counts):.3f}')

### Pauli-X measurement with Qiskit

In [ ]:
'''
# Circuit with a single qubit
qc = QuantumCircuit(1)

# Initialization of the circuit with random state
qc.initialize(psi.data, [0])

# Apply unitary to qubit state
qc.h(0)

# Measure Pauli-X operator
qc.measure_all()
qc.draw()
'''

In [ ]:
counts = execute(qc, simulator).result().get_counts()
plot_histogram(counts)

In [ ]:
print(f'Pauli-X measurement average for random state vector: {one_qubit_pauli_average_from_counts(counts):.3f}')

### Pauli-Y measurement with Qiskit

In [ ]:
'''
# Circuit with a single qubit
qc = QuantumCircuit(1)

# Initialization of the circuit with random state
qc.initialize(psi.data, [0])

# Apply unitary to qubit state
qc.sdg(0)
qc.h(0)

# Measure Pauli-Y operator
qc.measure_all()
qc.draw()
'''

In [ ]:
counts = execute(qc, simulator).result().get_counts()
plot_histogram(counts)

In [ ]:
print(f'Pauli-Y measurement average for random state vector: {one_qubit_pauli_average_from_counts(counts):.3f}')

### Spin measurement along an arbitrary axis with Qiskit

Suppose $\overrightarrow{v} = (v_x, v_y, v_z)$ is a three dimension unit vector with $\{v_x, v_y, v_z\} \in \Re$. We can define an observable

\begin{align}
\mathcal{O} = \overrightarrow{v} \cdot \overrightarrow{\sigma} = v_xX + v_yY + v_zZ
\end{align}

where $X$, $Y$ and $Z$ are the Pauli operators.

In [ ]:
'''
# Create a random real unitary vector
v = np.random.rand(3)
v /= np.linalg.norm(v)

# 3 circuits with a single qubit
qcs = [QuantumCircuit(1) for _ in range(3)]

# Initialization of the circuits with random state
[qc.initialize(psi.data, [0]) for qc in qcs]

# Apply unitary to qubit state
# Measurement along the X-axis
qcs[0].h(0)
# Measurement along the Y-axis
qcs[1].sdg(0)
qcs[1].h(0)
# Nothing to do for Z-axis

# Measure
[qc.measure_all() for qc in qcs]
'''

In [ ]:
# Job with bundled circuits
counts = execute(qcs, simulator).result().get_counts()

In [ ]:
# Average for X, Y and Z measurement respectively
avgs = list(map(one_qubit_pauli_average_from_counts, counts))
list(map(lambda v,s: v*s, v, avgs))

In [ ]:
print(f"Measurement average for random state vector along v-axis: {sum(map(lambda v,s: v*s, v, avgs)):.3f}")

## Measure a composite system

The state space of a composite physical system is the tensor product of the state spaces of the component physical systems.

For a system of $n$ qubits where qubit $i$ is in state $| \psi_i \rangle$, the state of the system is

\begin{align}
|\psi \rangle = |\psi_1 \rangle \otimes |\psi_2 \rangle \otimes \ldots \otimes |\psi_n \rangle 
\end{align}

We denote an operator $M$ acting on qubit $i$ of a system with $M_i$. For example, the Pauli string $Z_1Y_2X_3Z_4$ correspond to an operator where Pauli-$Z$ is applied to qubits 1 and 4, Pauli-$Y$ is applied to qubit 2 and Pauli-$X$ is applied to quibit 3.

Measurement of Pauli strings for composite systems will be important in studying the $H_2$ molecule. With this in mind, you have the task to implement the following method:

In [ ]:
def pauli_string_based_measurement(pauli_str: str) -> QuantumCircuit:
    """
    Creates a quantum circuit corresponding to the Pauli string passed 
    as a parameter.
    
    Parameters
    ----------
    pauli_str: str
        The Pauli string to measure with the quantum circuit.

        A Pauli string must be composed of the following characters:
          'X': measure the Pauli-X operator
          'Y': measure the Pauli-Y operator
          'Z': measure the Pauli-Z operator
          'I': measure the Identity operator
        
        This method is not case sensitive: 'XYZI', 'xyzi' and 'XyZi' are
        all valid strings.
        
        The length of the string determines the number of qubits in the
        quantum circuit. If `len(pauli_str)` is n, then the quantum
        circuit will have a n-qubits quantum register.
        
        To follow Qiskit convention, the string must be read from right to
        left. The last character of the string determines the Pauli
        operator to apply to qubit 0. The second to last character the
        Pauli operator to apply to qubit 1, etc. The first character of
        the string is for qubit (n-1)

        
    Raises
    ------
    ValueError
        If the the pauli_string passed as a parameter contains characters
        other that 'X','Y','Z' and 'I', or if the string is empty.
        
    """

    NotImplementedError()

In [ ]:
def pauli_average_from_counts(counts: Counts, pauli_str: str) -> float:
    """
    Computes the average value of a Pauli string from counts.
    
    Parameters
    ----------
    counts: Counts
        The counts that result from the execution of a job.
        
    pauli_str: str
        The Pauli string that was used to generate the circuit
        performing measurement in the computational basis.
        
    A Pauli string must be composed of the following characters:
          'X': measure the Pauli-X operator
          'Y': measure the Pauli-Y operator
          'Z': measure the Pauli-Z operator
          'I': measure the Identity operator
    
    This method is not case sensitive: 'XYZI', 'xyzi' and 'XyZi' are
    all valid strings.
        
    The length of the string determines the size of the basis set,
    hence the states present in the `counts` object.
            
    To follow Qiskit convention, the string must be read from right to
    left, which corresponds the bit ordering of the basis states.
    
    Raises
    ------
    ValueError
        If the the pauli_string passed as a parameter contains characters
        other that 'X','Y','Z' and 'I', or if the string is empty.
    """
    
    raise NotImplementedError()

### Typical workflow 

1. Setup a quantum circuit preparing some quantum state.
2. Identify a Pauli string representing the basis in which you want to perform the measurement and call your implementation of `pauli_string_based_measurement()`.
3. Append the measurement circuit to the circuit preparing the quantum state.
4. Execute the circuit and get the counts.
5. Call your `pauli_average_from_counts()` method to obtain the average.